In [1]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 980.9 kB/s eta 0:00:00
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached fsspec-2023.12.2-py3-none-any.whl.metadata (6.8 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvid

In [2]:
"""# Install the ultralytics package
pip install ultralytics"""
from ultralytics import YOLO  # Import the ModelYOLOv8 class from the yolov8 module
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

100%|██████████| 6.23M/6.23M [00:00<00:00, 14.9MB/s]


In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:

from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = Path(image_dir)
        self.label_dir = Path(label_dir)
        self.image_paths = list(self.image_dir.glob("*.jpg"))
        self.transform = transform
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label_path = self.label_dir / (image_path.stem + ".txt")
        image = Image.open(image_path).convert("RGB")
        label = self._parse_label(label_path)
        if self.transform:
            image = self.transform(image)
        return image, label
    def _parse_label(self, label_path):
        # Implement your label parsing logic here
        pass

In [ ]:
# Path to your custom dataset
train_image_dir = "/content/gdrive/My Drive/Colab Notebooks/ressource/yolo8_tp1/images/train"
train_label_dir = "/content/gdrive/My Drive/Colab Notebooks/ressource/yolo8_tp1/labels/train"
# Create an instance of the CustomDataset class
train_dataset = CustomDataset(train_image_dir, train_label_dir)

# Entraînez le modèle sur le dataset personnalisé
model.fit(train_dataset)

OSError: ignored

In [ ]:
from PIL import Image
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.image_paths = list(image_dir.glob("*.jpg"))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label_path = self.label_dir / (image_path.stem + ".txt")

        image = Image.open(image_path).convert("RGB")
        labels = self._parse_label(label_path)

        if self.transform:
            image = self.transform(image)

        return image, labels

    def _parse_label(self, label_path):
        with open(label_path, "r") as f:
            lines = f.readlines()

        labels = []
        for line in lines:
            label = line.strip().split(" ")
            labels.append(label)

        return labels

In [ ]:
ls '/content/gdrive/My Drive/Colab Notebooks/ressource/yolo8_tp1/images/val/000726.jpg'

In [ ]:
image_val = '/content/gdrive/My Drive/Colab Notebooks/ressource/yolov8_tp1/images/val/000726.jpg'

<img src="/content/gdrive/My Drive/Colab Notebooks/ressource/yolov8_tp1/images/val/000726.jpg" width=400 height=400 />

In [ ]:
image_val

In [ ]:
#results = model.predict('/content/gdrive/My Drive/Colab Notebooks/ressource/yolo8_tp1/images/val/000726.jpg', save=True, imgsz=320, conf=0.5)
classes =  4


# Detect objects in an image
results = model.predict('/content/gdrive/My Drive/Colab Notebooks/ressource/yolo8_tp1/images/val/000726.jpg', save=True, imgsz=320, conf=0.5, classes=classes)


In [ ]:
image_results = results[0]

In [ ]:
image_results

In [ ]:
# Accéder aux coordonnées des boîtes englobantes prédites
boxes = image_results.boxes.xyxy




In [ ]:
#scores = image_results.probs
scores='none'

In [ ]:
# Accéder aux classes prédites
#classes = image_results.probs.argmax(dim=1)
classes = 'none'

In [ ]:
boxes = boxes.long().tolist()

In [ ]:
boxes

In [ ]:
import cv2
import google.colab.patches as cv2_patches
# Charger l'image d'origine
image = cv2.imread('/content/gdrive/My Drive/Colab Notebooks/ressource/yolo8_tp1/images/val/000726.jpg')

# Parcourir les détections prédites
for box, score, cls in zip(boxes, scores, classes):
    # Dessiner la boîte englobante sur l'image
    x1, y1, x2, y2 = box
    #cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
    # Afficher le label et le score de confiance
    #label = f"{cls}: {score:.2f}"
    label = "pas de label"
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Afficher l'image avec les annotations prédites
#cv2_patches.cv2_imshow(image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/ressource/yolo8_tp1/000726.jpg', image)


In [ ]:
from IPython.core.interactiveshell import IPython
from IPython.display import Image
import IPython.display as display

In [ ]:
Image.open('/content/gdrive/My Drive/Colab Notebooks/ressource/yolo8_tp1/000726.jpg')

In [ ]:
import cv2
from PIL import Image

# Load the image
#image = cv2.imread('path/to/image.jpg')


image = cv2.imread(image_path)
plt.figure(figsize=(2, 2))
plt.imshow(image)
plt.show()